# Text Generation for Explaining Hamsters

In [ ]:
import json
import os
import sys

import boto3
import botocore

module_path = ".."
sys.path.append(os.path.abspath(module_path))


bedrock_runtime = boto3.client(
    service_name='bedrock-runtime', 
    aws_access_key_id=os.getenv('aws_access_key_id'),
    aws_secret_access_key=os.getenv('aws_secret_access_key'),
    region_name='us-west-2'
)

In [ ]:
scenario_description = """
Explain what hamsters are to 9th graders.
"""

In [ ]:
request_body = json.dumps({
    "inputText": "\n\nHuman: Explain what hamsters are to 9th graders.\n\nAssistant:",
    "textGenerationConfig":{
        "max_tokens_to_sample": 300,
        "temperature": 0.1,
        "top_p": 0.9,
        }
    }) 

In [ ]:
modelId = 'amazon.titan-tg1-large'
accept = 'application/json'
contentType = 'application/json'
generatedText = "\n"

model_response = bedrock_runtime.invoke_model(body=request_body, modelId=modelId, accept=accept, contentType=contentType)
response_contents = json.loads(model_response.get('body').read())

generatedText = response_contents.get('results')[0].get('outputText')

In [ ]:
model_response

In [ ]:
response_contents

In [ ]:
email_content = generatedText[generatedText.index('\n')+1:]
print(email_content)

# Streaming an Explanation of Hamsters

In [ ]:
output = []
streamed_response = bedrock_runtime.invoke_model_with_response_stream(body=request_body, modelId=modelId, accept=accept, contentType=contentType)
stream = streamed_response.get('body')

i = 1
if stream:
    try:
        async for event in stream:
            chunk = event.get('chunk')
            if chunk:
                chunk_obj = json.loads(chunk.get('bytes').decode())
                text = chunk_obj['outputText']
                output.append(text)
                print(f'\t\t\x1b[31m**Chunk {i}**\x1b[0m\n{text}\n')
                i += 1
    except Exception as e:
        print(f'Error processing stream: {str(e)}')

In [ ]:
print('\t\t\x1b[31m**COMPLETE OUTPUT**\x1b[0m\n')
complete_output = ''.join(output)
print(complete_output)